In [1]:
from __future__ import print_function
%matplotlib inline
import matplotlib.pylab as plt
import sys, os, glob
import numpy as np
import subprocess

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

from IPython.display import HTML
import xml.etree.ElementTree as ET
try:
    tree = ET.parse(os.environ['HADOOP_CONF_DIR'] + '/yarn-site.xml')
except IOError:
    raise IOError("Can't find the yarn configuration -- is HADOOP_CONF_DIR set?")
root = tree.getroot()
yarn_web_app = root.findall("./property[name='yarn.resourcemanager.webapp.address']")[0].find('value').text
yarn_web_app_string = "If this works successfully, you can check the <a target='_blank' href='http://{yarn_web_app}'>YARN application scheduler</a> and you should see your app listed there. Clicking on the 'Application Master' link will bring up the familiar Spark Web UI. "

plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size'] = 18
plt.style.use('fivethirtyeight')

## Set up and launch the Spark runtime

Remember from the previous notebook that we have a saved configuration in `./spark_config/` -- so all we need to do is set the `SPARK_CONF_DIR` environment variable and our default configuration will be used: 

In [2]:
# specify the configuration directory
os.environ['SPARK_CONF_DIR'] = os.path.realpath('../../spark_workshop/notebooks/twitter_dataframes/spark_config')

# how many cores do we have for the driver
ncores = int(os.environ.get('LSB_DJOB_NUMPROC', 1)) 

# here we set the memory we want spark to use for the driver JVM
os.environ['SPARK_DRIVER_MEMORY'] = '%dG'%(ncores*0.7)

# we have to tell spark which python executable we are using
os.environ['PYSPARK_PYTHON'] = subprocess.check_output('which python', shell=True).rstrip()

import findspark
findspark.init()

import pyspark
from pyspark import SparkConf, SparkContext

conf = SparkConf()

sc = SparkContext(master='yarn-client', conf=conf)

## Initialization

Using `DataFrame`s, our entry point into the Spark universe is the `SQLContext` or the equivalent `HiveContext`:

In [3]:
from pyspark.sql import SQLContext, HiveContext

hc = HiveContext(sc)

We'll load the data off the disk, but only for the last three months of the year. 

In [4]:
%%time
data = hc.read.parquet('/user/roskarr/twitter/2014_1*')

CPU times: user 15 ms, sys: 3 ms, total: 18 ms
Wall time: 45.4 s


`data` is now a `DataFrame` object, which is essentially a collection of `Row` objects. Each `Row` object contains data for whatever columns are defined in the `DataFrame`. Here is a critical difference between `DataFrames` and `RDD`s: each column has an associated data type. While in dealing with an `RDD` we relied on Python to convert types, here each column has a specified data type. This means that:

a) we have to be a bit more careful about what we are doing and 

b) the execution engine can optimize our calculations because the data is no longer a black box. 

### Importing libraries and types for `DataFrame` API

The next few cells are a bit lengthy and complicated so you can just treat them as black boxes for now. First, we just import some necessary libraries and classes, then a series of functions are defined and executed on the `data` to give us a `DataFrame` of hashtags. 

In [5]:
import pyspark.sql.functions as func
from pyspark.sql import Row, Window
from pyspark.sql.types import IntegerType, ArrayType, StringType, NullType, LongType, StructField, StructType, DateType, DataType, DateConverter, DatetimeConverter, TimestampType, BooleanType
import datetime

### Custom functions to extract hashtags

Change the date format to YYYY-MM-DD HH:MM:SS

In [6]:
convert_date_string = func.udf(lambda date_string: \
                               datetime.date.strftime(datetime.datetime.strptime(date_string, \
                                                                                 '%a %b %d %H:%M:%S +0000 %Y'),\
                                                      '%Y-%m-%d %H:%M:%S'), StringType())

Convert the date string to a datetime object

In [7]:
datetime_udf = func.udf(lambda date_string: datetime.strptime(date_string, '%a %b %d %H:%M:%S +0000 %Y'), DateType())

Extract the hashtags from a hashtag array and convert them all to lowercase

In [8]:
hash_text_udf = func.udf(lambda row: [r.text.lower() for r in row], returnType=ArrayType(StringType()))

Convert the tweet text string to lowercase

In [9]:
key_udf_lower = func.udf(lambda row: row.lower(), returnType=StringType())

Split the text of the tweet

In [10]:
key_udf_split = func.udf(lambda row: row.split(), returnType=ArrayType(StringType()))

Combine the year of creation of the tweet and the day of creation to a unique date ID.

In [11]:
new_date = (lambda col: func.dayofyear(col) + func.year(col)*1000)

### Apply the functions to the data frame

Define the number of partitions for the data

In [26]:
Npartitions = sc.defaultParallelism*10
print(Npartitions)

800


Create the hashtag dataframe with all the tweets with more than one hashtag, extract the hashtags and also extract the creation date and convert it to string.

In [27]:
# only keep the tweets with at least one hashtag
hashtag_df = (data.select('created_at', 'entities.hashtags', 'text')
                .filter(func.size('hashtags') > 1)
                .withColumn('hash_text', hash_text_udf('hashtags'))
                .withColumn('created_at', convert_date_string('created_at'))
                .repartition(Npartitions))
hashtag_df.cache()

DataFrame[created_at: string, hashtags: array<struct<indices:array<bigint>,text:string>>, text: string, hash_text: array<string>]

In [28]:
hashtag_df.show(10)

+-------------------+--------------------+--------------------+--------------------+
|         created_at|            hashtags|                text|           hash_text|
+-------------------+--------------------+--------------------+--------------------+
|2014-10-02 18:53:08|[[WrappedArray(40...|Que esa frase me ...|[guerreroencanalf...|
|2014-10-04 00:15:00|[[WrappedArray(41...|RT @dafnehurley: ...|[rt, fav, dinámicas]|
|2014-10-04 21:39:47|[[WrappedArray(62...|秋のお出かけなら【登別温泉】へ行き...|    [北海道, 旅行, 秋, 紅葉]|
|2014-10-06 10:47:52|[[WrappedArray(91...|RT @Asr3Follow01:...|[ريتويت, تابعني_ا...|
|2014-10-08 00:28:58|[[WrappedArray(49...|I got a reward: L...|[ipad, gameinsigh...|
|2014-10-09 15:40:24|[[WrappedArray(34...|RT @fayez_1977: ب...|[الهلال, رتويت, ا...|
|2014-10-11 03:49:06|[[WrappedArray(0,...|#ArgentinaNeedSha...|[argentinaneedsha...|
|2014-10-12 16:14:16|[[WrappedArray(35...|RT @A1Mufasa: sup...|[iwaju, iwatago, ...|
|2014-10-14 01:40:16|[[WrappedArray(0,...|#Video #Demonstra...|[v

Convert the tweet text to lowercase, split it into a word array and add this into a new column called test. Then check if any of the elements of keys is in this list and add this information in the column intext.

In [ ]:
def testfunc(row, keys):
    result_tmp = [True for val in keys]
#     result_tmp = np.array(result_tmp, dtype=np.int)    
    return result_tmp

In [29]:
keys = ['@tom_wilso:', 'rt']
# key_udf_key = func.udf(lambda row: len([val for val in keys if val in row]) > 0, returnType=BooleanType())
key_udf_global3 = func.udf(lambda row: [int(val in row) for val in keys], returnType=ArrayType(IntegerType()))

In [45]:
hashtag_df = hashtag_df.withColumn('test', key_udf_lower('text'))
hashtag_df = hashtag_df.withColumn('test', key_udf_split('test'))
hashtag_df.show(20)

+-------------------+--------------------+--------------------+--------------------+--------------------+
|         created_at|            hashtags|                text|           hash_text|                test|
+-------------------+--------------------+--------------------+--------------------+--------------------+
|2014-10-02 18:53:08|[[WrappedArray(40...|Que esa frase me ...|[guerreroencanalf...|[que, esa, frase,...|
|2014-10-04 00:15:00|[[WrappedArray(41...|RT @dafnehurley: ...|[rt, fav, dinámicas]|[rt, @dafnehurley...|
|2014-10-04 21:39:47|[[WrappedArray(62...|秋のお出かけなら【登別温泉】へ行き...|    [北海道, 旅行, 秋, 紅葉]|[秋のお出かけなら【登別温泉】へ行...|
|2014-10-06 10:47:52|[[WrappedArray(91...|RT @Asr3Follow01:...|[ريتويت, تابعني_ا...|[rt, @asr3follow0...|
|2014-10-08 00:28:58|[[WrappedArray(49...|I got a reward: L...|[ipad, gameinsigh...|[i, got, a, rewar...|
|2014-10-09 15:40:24|[[WrappedArray(34...|RT @fayez_1977: ب...|[الهلال, رتويت, ا...|[rt, @fayez_1977:...|
|2014-10-11 03:49:06|[[WrappedArray(0,...|#Arg

In [46]:
keys = ['@oridiva:', 'rt']
# key_udf_key = func.udf(lambda row: len([val for val in keys if val in row]) > 0, returnType=BooleanType())
key_udf_global3 = func.udf(lambda row: [int(val in row) for val in keys], returnType=ArrayType(IntegerType()))
key_udf_global4 = func.udf(lambda row: int('@oridiva:' in row), returnType=IntegerType())
key_udf_global5 = func.udf(lambda row: int('rt' in row), returnType=IntegerType())

hashtag_df5 = (hashtag_df.withColumn('@oridiva:', key_udf_global4('test'))
                         .withColumn('rt', key_udf_global5('test')))

In [47]:
hashtag_df5.show(10)

+-------------------+--------------------+--------------------+--------------------+--------------------+---------+---+
|         created_at|            hashtags|                text|           hash_text|                test|@oridiva:| rt|
+-------------------+--------------------+--------------------+--------------------+--------------------+---------+---+
|2014-10-02 18:53:08|[[WrappedArray(40...|Que esa frase me ...|[guerreroencanalf...|[que, esa, frase,...|        0|  0|
|2014-10-04 00:15:00|[[WrappedArray(41...|RT @dafnehurley: ...|[rt, fav, dinámicas]|[rt, @dafnehurley...|        0|  1|
|2014-10-04 21:39:47|[[WrappedArray(62...|秋のお出かけなら【登別温泉】へ行き...|    [北海道, 旅行, 秋, 紅葉]|[秋のお出かけなら【登別温泉】へ行...|        0|  0|
|2014-10-06 10:47:52|[[WrappedArray(91...|RT @Asr3Follow01:...|[ريتويت, تابعني_ا...|[rt, @asr3follow0...|        0|  1|
|2014-10-08 00:28:58|[[WrappedArray(49...|I got a reward: L...|[ipad, gameinsigh...|[i, got, a, rewar...|        0|  0|
|2014-10-09 15:40:24|[[WrappedArray(34..

In [18]:
hashtag_df2 = hashtag_df.withColumn('intext', key_udf_global3('test'))
hashtag_df2.show(10)

+-------------------+--------------------+--------------------+--------------------+--------------------+------+
|         created_at|            hashtags|                text|           hash_text|                test|intext|
+-------------------+--------------------+--------------------+--------------------+--------------------+------+
|2014-10-01 06:28:46|[[WrappedArray(36...|RT @sougo_kizuna:...|[sougofollow, fol...|[rt, @sougo_kizun...|[0, 1]|
|2014-10-02 07:33:36|[[WrappedArray(3,...|tj #kcaargentina ...|[kcaargentina, lali]|[tj, #kcaargentin...|[0, 0]|
|2014-10-02 19:25:43|[[WrappedArray(13...|RT @novruss: #Нов...|[новороссия, луга...|[rt, @novruss:, #...|[0, 1]|
|2014-10-03 15:40:35|[[WrappedArray(80...|Я собрал 65 700 з...|[ipad, ipadgames,...|[я, собрал, 65, 7...|[0, 0]|
|2014-10-04 06:09:24|[[WrappedArray(93...|RT @HomeAdore: Ei...|[architecture, in...|[rt, @homeadore:,...|[1, 1]|
|2014-10-04 21:33:22|[[WrappedArray(15...|RT @tom_wilso: #N...|  [nats, nats, nats]|[rt, @tom_wi

Compute the day ID of the tweet and add it to the column new_date

In [48]:
hashtag_df3 = hashtag_df5.withColumn('new_date', new_date('created_at'))

In [49]:
hashtag_df3.show(10)

+-------------------+--------------------+--------------------+--------------------+--------------------+---------+---+--------+
|         created_at|            hashtags|                text|           hash_text|                test|@oridiva:| rt|new_date|
+-------------------+--------------------+--------------------+--------------------+--------------------+---------+---+--------+
|2014-10-02 18:53:08|[[WrappedArray(40...|Que esa frase me ...|[guerreroencanalf...|[que, esa, frase,...|        0|  0| 2014275|
|2014-10-04 00:15:00|[[WrappedArray(41...|RT @dafnehurley: ...|[rt, fav, dinámicas]|[rt, @dafnehurley...|        0|  1| 2014277|
|2014-10-04 21:39:47|[[WrappedArray(62...|秋のお出かけなら【登別温泉】へ行き...|    [北海道, 旅行, 秋, 紅葉]|[秋のお出かけなら【登別温泉】へ行...|        0|  0| 2014277|
|2014-10-06 10:47:52|[[WrappedArray(91...|RT @Asr3Follow01:...|[ريتويت, تابعني_ا...|[rt, @asr3follow0...|        0|  1| 2014279|
|2014-10-08 00:28:58|[[WrappedArray(49...|I got a reward: L...|[ipad, gameinsigh...|[i, got, a, r

In [50]:
hashtag_df4 = hashtag_df3.select('new_date', '@oridiva:', 'rt')

In [52]:
hashtag_df4.show(20)

+--------+---------+---+
|new_date|@oridiva:| rt|
+--------+---------+---+
| 2014275|        0|  0|
| 2014277|        0|  1|
| 2014277|        0|  0|
| 2014279|        0|  1|
| 2014281|        0|  0|
| 2014282|        0|  1|
| 2014284|        0|  0|
| 2014285|        0|  1|
| 2014287|        0|  0|
| 2014288|        0|  0|
| 2014289|        0|  1|
| 2014291|        0|  0|
| 2014292|        0|  1|
| 2014293|        0|  0|
| 2014294|        0|  1|
| 2014296|        0|  1|
| 2014297|        0|  0|
| 2014298|        0|  0|
| 2014299|        1|  1|
| 2014301|        0|  0|
+--------+---------+---+
only showing top 20 rows



In [53]:
hashtag_df4_group = hashtag_df4.groupBy('new_date')

In [54]:
hashtag_df4_group.sum('@oridiva:').collect()

Py4JJavaError: An error occurred while calling o402.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 34 in stage 47.0 failed 4 times, most recent failure: Lost task 34.3 in stage 47.0 (TID 8267, a3148.hpc-net.ethz.ch): java.lang.OutOfMemoryError: unable to create new native thread
	at java.lang.Thread.start0(Native Method)
	at java.lang.Thread.start(Thread.java:691)
	at org.apache.spark.api.python.PythonRunner.compute(PythonRDD.scala:120)
	at org.apache.spark.sql.execution.BatchPythonEvaluation$$anonfun$doExecute$1.apply(python.scala:398)
	at org.apache.spark.sql.execution.BatchPythonEvaluation$$anonfun$doExecute$1.apply(python.scala:363)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$20.apply(RDD.scala:710)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$20.apply(RDD.scala:710)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:38)
	at org.apache.spark.rdd.RDD.computeOrReadCheckp

Questions:
* What does the text method do?
* Why does the new_date function work even though it is not an UDF?